In [ ]:
from os import environ

environ['optimizer'] = 'Adam'
environ['num_workers']= '8'
environ['batch_size']= '8192'
environ['n_epochs']= '1500'
environ['batch_norm']= 'True'
environ['loss_func']='MAPE'
environ['layers'] = '600 350 200 180'
environ['dropouts'] = '0.3 '*4
environ['log'] = 'False'
environ['weight_decay'] = '0.01'
environ['cuda_device'] ='cuda:0'
environ['dataset'] = 'speedup_dataset_augm_600.pkl'

%run utils.ipynb

In [ ]:
print("loading data")

train_dl, val_dl, test_dl = train_dev_split(dataset,val_size=10000, test_size=10000, batch_size=batch_size, num_workers=num_workers, log=log)
# train_dl, val_dl, test_dl = train_dev_split_filter(dataset,val_size=10000, test_size=10000, batch_size=batch_size, num_workers=num_workers, log=log, speedup_lo_bound=0.2)
db = fai.basic_data.DataBunch(train_dl, val_dl, test_dl, device=device)

print("data loaded")
print(val_dl.dataset.X.shape)

In [ ]:
input_size = train_dl.dataset.X.shape[1]
output_size = train_dl.dataset.Y.shape[1]
print(train_dl.dataset.X.shape)
print(train_dl.dataset.Y)

model = None 

if batch_norm:
    model = Model_BN_LeakyReLU(input_size, output_size, hidden_sizes=layers_sizes, drops=drops)
else:
    model = Model(input_size, output_size)

# model = nn.DataParallel(model)
# model.to(device)

if loss_func == 'MSE':
    criterion = nn.MSELoss()
else:
    criterion = mape_criterion

l = fai.basic_train.Learner(db, model, loss_func=criterion, metrics=[mape_criterion, rmse_criterion],
                            callback_fns=[partial(EarlyStoppingCallback, mode='min', 
                            monitor='mape_criterion', min_delta=0, patience=150)],silent=True)

if optimizer == 'SGD':
    l.opt_func = optim.SGD 
    

In [ ]:
l.lr_find()
l.recorder.plot()

In [ ]:
lr = 1e-02

In [ ]:
l.fit_one_cycle(1500, lr)

In [ ]:
l.recorder.plot_losses()

In [ ]:
l.save(f"speedup_{optimizer}_batch_norm_{batch_norm}_{loss_func}_nlayers_{len(layers_sizes)}_log_{log}_LeakyReLU")

In [ ]:
val_df = get_results_df(val_dl, l.model)
train_df = get_results_df(train_dl, l.model)
test_df=get_results_df(test_dl, l.model)

In [ ]:
test_df.describe()

In [ ]:
val_df.describe()

In [ ]:
train_df.describe()

In [ ]:
gdf_t=pd.DataFrame(train_df.groupby(['name']).mean())
gdf_v=pd.DataFrame(val_df.groupby(['name']).mean())

In [ ]:
label=list(gdf_v.index)
mape=gdf_v['APE']
index = np.arange(len(label))
plt.figure(figsize=(20,10))
plt.bar(index, mape)
plt.xlabel('function', fontsize=5)
plt.ylabel('mape', fontsize=5)
plt.xticks(index, label, fontsize=5, rotation=30)
plt.title('-')
plt.show()

In [ ]:
train_df.groupby(['name']).mean().head()

In [ ]:
df[(df.interchange==0) & (df.unroll == 0) & (df.tile == 0)][['prediction','target', 'abs_diff','APE']]

In [ ]:
df1 = df[(df.interchange==0) & (df.unroll == 0) & (df.tile == 0)]
#joint_plot(df1, f"Validation dataset, {loss_func} loss")
df2 = df
joint_plot(df2, f"Validation dataset, {loss_func} loss")

In [ ]:
df_ = df.sort_values(by=["APE"])

df_['x'] = range(len(df_))

In [ ]:
plt.plot('x', 'APE', 'bo', data=df_)


plt.xlabel('scheduled program')
plt.ylabel('APE')
plt.legend()

In [ ]:
plt.plot('x', 'APE', 'go', data=df_)

In [ ]:
l = l.load(f"speedup_{optimizer}_batch_norm_{batch_norm}_{loss_func}_nlayers_{len(layers_sizes)}_log_{log}_aumen600_leakyReLU")